In [45]:
import speech_recognition as srec
from gtts import gTTS
import pyttsx3 as pyt
import os

from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, VitsModel
import torch

import requests
import json

import time
import psutil

import whisper
import sounddevice as sd
import numpy as np
from IPython.display import Audio

In [3]:
engine = pyt.init()
voices = engine.getProperty('voices')
engine.setProperty('voice', voices[1].id)

device = "auto"

In [4]:
# --- AZURE SAILOR 0.5B MODEL ---
start_time = time.time()

model = AutoModelForCausalLM.from_pretrained(
    'sail/Sailor-0.5B-Chat',
    torch_dtype="auto",
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained('sail/Sailor-0.5B-Chat')
end_time = time.time()
load_time = end_time - start_time
print(load_time)


c:\Users\kimbe\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\kimbe\.cache\huggingface\hub\models--sail--Sailor-0.5B-Chat. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Some parameters are on the meta device because they were offloaded to the disk and cpu.


566.1801528930664


In [16]:
# --- LLAMA ---

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-1B")
model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-3.2-1B")

In [46]:
# --- STT ---

# def perintah():
#     mendengar = srec.Recognizer()
#     with srec.Microphone() as source:
#         print('Mendengarkan......')
#         suara = mendengar.listen(source, phrase_time_limit=5)
#         try:
#             print('Diterima.....')
#             dengar = mendengar.recognize_google(suara, language='id-ID')
#             print(dengar)
#         except:
#             pass
#         return dengar

whisper_model = whisper.load_model("small")

def perintah():
    duration = 5
    sample_rate = 16000  

    print("Mendengarkan......")
    audio_data = sd.rec(int(duration * sample_rate), samplerate=sample_rate, channels=1, dtype='float32')
    sd.wait()  
    print("Diterima.....")

    audio_data = np.squeeze(audio_data)  
    dengar = whisper_model.transcribe(audio_data, fp16=False, language="id")
    print(dengar["text"])

    return dengar["text"]

In [55]:
# --- TTS ---

# def ngomong(text):
#     voices = engine.getProperty('voices')

#     for voice in voices:
#         if "MSTTS_V110_idID_Andika" in voice.id:
#             engine.setProperty('voice', voice.id)
#             break

#     # Speak the text
#     engine.say(text)
    
#     # Wait until speaking is finished
#     engine.runAndWait()

mms = VitsModel.from_pretrained("facebook/mms-tts-ind")
mms_token = AutoTokenizer.from_pretrained("facebook/mms-tts-ind")

def ngomong(text):
    inputs = mms_token(text, return_tensors="pt")

    with torch.no_grad():
        output = mms(**inputs).waveform
    
    return Audio(output.squeeze().cpu().numpy(), rate=16000) # Can change rate to make it faster/slower

In [21]:
def run_va():
    # prompt from user
    Layanan = perintah()

    if not Layanan:
        print("Tidak ada input yang terdeteksi. Silakan coba berbicara lagi.")
        return

    # messages = [{"role": "user", "content": Layanan}]
    # prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

    # outputs = pipeline(prompt, max_new_tokens=256, do_sample=True, temperature=0.7, top_k=50, top_p=0.95)
    # print(outputs[0]["generated_text"])

    # ngomong(Layanan)

    # print(Layanan)

    system_prompt= 'Jawab dalam bahasa indonesia.'

    messages = [
        {"role": "question", "content": Layanan}
        ]

    # text = tokenizer.apply_chat_template(
    #     messages,
    #     tokenize=False,
    #     add_generation_prompt=True
    # )

    model_inputs = tokenizer(Layanan, return_tensors="pt") 

    start_time = time.time()
    memory_before = psutil.virtual_memory().used

    generated_ids = model.generate(
        model_inputs.input_ids,
        max_new_tokens=512, 
        temperature=0.7,
        top_p=0.9

    )

    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]
    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

    end_time = time.time()
    memory_after = psutil.virtual_memory().used

    inference_time = end_time - start_time
    memory_used = memory_after - memory_before
    cpu_usage = psutil.cpu_percent(interval=1)
    
    print(response)
    print("INFERENCE TIME: ", inference_time)
    print("MEMORY USAGE: ", memory_used)
    print("CPU USAGE: ", cpu_usage)
    ngomong(response)

In [56]:
run_va()

Mendengarkan......
Diterima.....
 siapa jokowi dodoh?
 klo dia jokowi noh, berarti dia jokowi 10 tahun yang lalu. Karena sejak saat itu, Joko Widodo memilih memakai pakaian seragam sekolah dan juga memakai topi tradisional orang Betawi di mana ia berasal.

Begitupun dengan Soerjo, meskipun ia terlahir sebagai warga negara Indonesia namun ia mengaku tidak pernah melihat Presiden RI pertama, Mohammad Hatta (mantan Presiden Indonesia) atau Soekarno (mantan Presiden Indonesia) memakai topi asli orang Betawi. Dan hanya kali ini, Soerjo mendapatkan kesempatan untuk berfoto bersama Soerjowiyono yang merupakan mantan siswi SMA Betawi yang telah menjadi selebriti nasional. Sebab, setiap ada pembukaan Asian Games 2018, akan ditampilkan foto Soerjo yang membawa topi tradisional orang Betawi. Bahkan Soerjo juga sempat mengabadikan momen itu sambil tersenyum simpul.

Kedua orang tua Soerjo meninggal pada tahun 2009. Ketika itu, Soerjo sangat sedih karena sudah menyaksikan sang ayah meninggal dunia.